In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

In [ ]:
from IPython.display import Image, display

def imshow(img, ax=None):
	if ax is None:
		ret, encoded = cv2.imencode(".jpg", img)
		display(Image(encoded))
	else:
		ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
		ax.axis('off')

### Part 1 a

In [ ]:
img = cv2.imread("three.jpg") # Read image 

# Setting parameter values 
t_lower = 30 # Lower Threshold 
t_upper = 150 # Upper threshold 

# Applying the Canny Edge filter 
edge = cv2.Canny(img, t_lower, t_upper) 

imshow(img)
imshow(edge)
#cv2.imshow('original', img) 
#cv2.imshow('edge', edge) 
#cv2.waitKey(0) 
#cv2.destroyAllWindows() 

In [ ]:
import cv2
import numpy as np
from IPython.display import Image, display
from matplotlib import pyplot as plt

#Image loading
img = cv2.imread("four.jpg")
# Show image
imshow(img)

#image grayscale conversion
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imshow(gray)

#Threshold Processing
ret, bin_img = cv2.threshold(gray,
							0, 255, 
							cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
imshow(bin_img)


# noise removal
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
bin_img = cv2.morphologyEx(bin_img, 
						cv2.MORPH_CLOSE,
						kernel,
						iterations=27)
imshow(bin_img)


# Create subplots with 1 row and 2 columns
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(8, 8))
# sure background area
sure_bg = cv2.dilate(bin_img, kernel, iterations=3)
imshow(sure_bg, axes[0,0])
axes[0, 0].set_title('Sure Background')

# Distance transform
dist = cv2.distanceTransform(bin_img, cv2.DIST_L2, 5)
imshow(dist, axes[0,1])
axes[0, 1].set_title('Distance Transform')

#foreground area
ret, sure_fg = cv2.threshold(dist, 0.5 * dist.max(), 255, cv2.THRESH_BINARY)
sure_fg = sure_fg.astype(np.uint8) 
imshow(sure_fg, axes[1,0])
axes[1, 0].set_title('Sure Foreground')

# unknown area
unknown = cv2.subtract(sure_bg, sure_fg)
imshow(unknown, axes[1,1])
axes[1, 1].set_title('Unknown')

plt.show()


# Marker labelling
# sure foreground 
ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that background is not 0, but 1
markers += 1
# mark the region of unknown with zero
markers[unknown == 255] = 0

fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(markers, cmap="tab20b")
ax.axis('off')
plt.show()


# watershed Algorithm
markers = cv2.watershed(img, markers)

fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(markers, cmap="tab20b")
ax.axis('off')
plt.show()


labels = np.unique(markers)

coins = []
for label in labels[2:]: 

# Create a binary image in which only the area of the label is in the foreground 
#and the rest of the image is in the background 
	target = np.where(markers == label, 255, 0).astype(np.uint8)

# Perform contour extraction on the created binary image
	contours, hierarchy = cv2.findContours(
		target, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
	)
	coins.append(contours[0])

# Draw the outline
img = cv2.drawContours(img, coins, -1, color=(0, 23, 223), thickness=2)
imshow(img)



In [ ]:
import numpy as np
import cv2

img = cv2.imread("two.jpg")
img = cv2.resize(img , (640 , 800))
image_copy = img.copy()
img = cv2.GaussianBlur(img , (7 , 7) , 3)

gray = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY)
ret , thresh = cv2.threshold(gray , 170 , 255 , cv2.THRESH_BINARY)

contours , _ = cv2.findContours(thresh , cv2.RETR_TREE , cv2.CHAIN_APPROX_NONE)
area = {}
for i in range(len(contours)):
    cnt = contours[i]
    ar = cv2.contourArea(cnt)
    area[i] = ar
srt = sorted(area.items() , key = lambda x : x[1] , reverse = True)
results = np.array(srt).astype("int")
num = np.argwhere(results[: , 1] > 500).shape[0]

for i in range(1 , num):
    image_copy = cv2.drawContours(image_copy , contours , results[i , 0] ,
                                  (0 , 255 , 0) , 3)
print("Number of coins is " , num - 1)

imshow(image_copy)

In [ ]:
def detect_coins(image_path):
    """
    Detect coins in the image using edge detection.
    
    Args:
        image_path (str): Path to the input image
        
    Returns:
        tuple: (original image, image with detected coins outlined)
    """
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Could not read the image")
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (11, 11), 0)
    
    # Apply Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)
    
    # Find contours
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a copy of the original image for visualization
    result = image.copy()
    
    # Draw contours on the image
    cv2.drawContours(result, contours, -1, (0, 255, 0), 2)
    
    return image, result, contours

In [ ]:
def segment_coins(image, contours):
    """
    Segment individual coins from the image.
    
    Args:
        image (numpy.ndarray): Original image
        contours (list): List of detected contours
        
    Returns:
        list: List of segmented coin images
    """
    segmented_coins = []
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    
    for contour in contours:
        # Create a mask for each coin
        cv2.drawContours(mask, [contour], -1, 255, -1)
        
        # Extract the coin using the mask
        coin = cv2.bitwise_and(image, image, mask=mask)
        
        # Get bounding rectangle
        x, y, w, h = cv2.boundingRect(contour)
        
        # Crop the coin
        coin_cropped = coin[y:y+h, x:x+w]
        
        # Only add if the coin is large enough (to filter out noise)
        if w > 30 and h > 30:
            segmented_coins.append(coin_cropped)
        
        # Clear mask for next coin
        mask.fill(0)
    
    return segmented_coins

In [ ]:
def count_coins(contours, min_area=1000):
    """
    Count the total number of coins in the image.
    
    Args:
        contours (list): List of detected contours
        min_area (int): Minimum area to consider as a coin
        
    Returns:
        int: Number of coins detected
    """
    # Filter contours based on area to remove noise
    valid_coins = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]
    return len(valid_coins)


In [ ]:
def process_image(image_path):
    """
    Main function to process the image and display results.
    
    Args:
        image_path (str): Path to the input image
    """
    try:
        # Detect coins
        original, detected, contours = detect_coins(image_path)
        
        # Segment coins
        segmented_coins = segment_coins(original, contours)
        
        # Count coins
        coin_count = count_coins(contours)
        
        # Display original and detected images
        plt.figure(figsize=(12, 4))
        
        # Original image
        plt.subplot(1, 3, 1)
        plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        plt.title('Original Image')
        plt.axis('off')
        
        # Detected coins
        plt.subplot(1, 3, 2)
        plt.imshow(cv2.cvtColor(detected, cv2.COLOR_BGR2RGB))
        plt.title('Detected Coins')
        plt.axis('off')
        
        # Combined segmented image
        plt.subplot(1, 3, 3)
        combined = np.zeros_like(original)
        for contour in contours:
            if cv2.contourArea(contour) > 1000:  # Only draw large contours
                # Fill with a random color
                color = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
                cv2.drawContours(combined, [contour], -1, color, -1)
        plt.imshow(cv2.cvtColor(combined, cv2.COLOR_BGR2RGB))
        plt.title(f'Segmented Coins (Count: {coin_count})')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Display segmented coins in a separate figure if there are any
        if segmented_coins:
            # Calculate grid dimensions
            n = len(segmented_coins)
            cols = 4  # Fixed number of columns
            rows = (n + cols - 1) // cols  # Ceiling division
            
            plt.figure(figsize=(12, 3*rows))
            plt.suptitle('Individual Segmented Coins', fontsize=16)
            
            for i, coin in enumerate(segmented_coins):
                plt.subplot(rows, cols, i+1)
                plt.imshow(cv2.cvtColor(coin, cv2.COLOR_BGR2RGB))
                plt.title(f'Coin {i+1}')
                plt.axis('off')
            
            plt.tight_layout()
            plt.subplots_adjust(top=0.9)  # Make room for the main title
            plt.show()
        
        print(f"Total number of coins detected: {coin_count}")
        
    except Exception as e:
        print(f"Error processing image: {str(e)}")
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)

In [ ]:
if __name__ == "__main__":
    # Replace with your image path
    image_path = "./five.jpg"
    process_image(image_path)